In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import makedirs
from requests import get
from matplotlib import rcParams
from blaseball_mike.database import get_all_teams

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

rcParams.update({'figure.autolayout': True})

In [2]:
# Set to folder path
base_url = "https://www.blaseball.com/database/teamElectionStats?id="

teams = get_all_teams()
wills_files = {'_'.join(teams[team]['nickname'].lower().split(
)): team for team in teams if teams[team]['stadium'] is not None}

will_names = {
    'roster_swap': 'Swap',
    'equivalent_exchange': 'Exchange',
    'mod_reroll': 'Reform',
    'shadow_infuse': 'Shadow Infuse',
    'player_move': 'Move',
    'shadow_revoke': 'Shadow Revoke',
    'magnify': 'Magnify',
    'item_steal': 'Take Item'
}

wills_votes = [pd.DataFrame(get(f'{base_url}{wills_files[key]}').json()[
                            'wills']) for key in wills_files]

In [3]:
magic_url = base_url + '7966eb04-efcc-499b-8f03-d13916330531'

In [4]:
base_url + list(wills_files.values())[0]

'https://www.blaseball.com/database/teamElectionStats?id=a37f9158-7f82-46bc-908c-c9e2dda7c33b'

In [5]:
for team in wills_files:
    try:
        makedirs(f'./plots/{team}')
    except:
        pass
    try:
        makedirs(f'./data/{team}')
    except:
        pass

In [6]:
wills_values = {key: None for key in wills_files}

for i, key in enumerate(wills_files):
    wills_votes[i]['percent'] = wills_votes[i]['percent'].map(lambda x: float(x))

    if wills_votes[i]['percent'].sum() != 1:
        wills_votes[i].loc[wills_votes[i]['percent'] == wills_votes[i]['percent'].max(
        ), 'percent'] += round(100*(1 - wills_votes[i]['percent'].sum()/100),1)
        
    wills_votes[i]['percent'] = wills_votes[i]['percent'].map(lambda x: round(float(x),1))
    
    wills_votes[i]['team'] = key
    wills_votes[i]['will'] = wills_votes[i]['id'].map(will_names)

    wills_values[key] = {
        will: odds for will, odds in zip(wills_votes[i]['will'], wills_votes[i]['percent'])
    }

In [7]:
wills_values['magic']

{'Exchange': 10.1,
 'Take Item': 6.1,
 'Magnify': 25.2,
 'Reform': 14.0,
 'Move': 19.7,
 'Swap': 5.3,
 'Shadow Infuse': 18.6,
 'Shadow Revoke': 1.0}

In [8]:
def draw_wills(wills):
    ps = pd.Series(list(wills.values()))/100
    return np.random.choice(list(wills.keys()), 2, False, ps)

In [9]:
def draw_free_wills(wills):
    ps = pd.Series(list(wills.values()))/100
    return np.random.choice(list(wills.keys()), 3, False, ps)

In [10]:
def will_sim(will_dict, n_draws=50_000):

    draws = {
        'will_1': [],
        'will_2': []
    }

    for i in range(1, n_draws+1):
        wills = draw_wills(will_dict)
        wills.sort()
        draws['will_1'].append(wills[0])
        draws['will_2'].append(wills[1])

        if i % (n_draws // 4) == 0:
            print(f'Simulation {i}...')

    return pd.DataFrame(draws)

In [11]:
def free_will_sim(will_dict, n_draws=50_000):
    draws = {
        'will_1': [],
        'will_2': [],
        'will_3': []
    }

    for i in range(1, n_draws+1):
        wills = draw_free_wills(will_dict)
        wills.sort()
        draws['will_1'].append(wills[0])
        draws['will_2'].append(wills[1])
        draws['will_3'].append(wills[2])

        if i % (n_draws // 4) == 0:
            print(f'Simulation {i}...')

    return pd.DataFrame(draws)

In [12]:
def get_tables(draws):
    will_1_freq = pd.Series(draws.groupby('will_1').size()) / draws.shape[0]
    will_1_freq = will_1_freq.sort_values(ascending=False)
    will_2_freq = pd.Series(draws.groupby('will_2').size()) / draws.shape[0]
    will_2_freq = will_2_freq.sort_values(ascending=False)
    
    wills = pd.DataFrame(draws.groupby(['will_1','will_2'])[
                         'will_3'].value_counts(normalize=True))
        
    wills.columns = ['prob']
    wills = wills.sort_values('prob', ascending=False)
        
    wills['will1_prob'] = wills.index.map(lambda x: will_1_freq[x[0]])
    wills['will2_prob'] = wills.index.map(lambda x: will_2_freq[x[1]])

    wills['relative_prob'] = wills['prob'] * wills['will1_prob'] * wills['will2_prob']
    
    wills = wills.sort_values('relative_prob', ascending=False).drop(
        columns=['prob', 'will1_prob','will2_prob'])
    wills.reset_index(inplace=True)
    wills['will_combo'] = wills['will_1'] + ' + ' + wills['will_2'] + ' + ' + wills['will_3']

    wills.to_csv('./plots/magic/test.csv')

    
    overall_probs = {}
    for outcome in set(wills['will_1']).union(set(wills['will_2'])).union(set(wills['will_3'])):
        overall_probs[outcome] = round(wills[(wills['will_1'] == outcome) | (
            wills['will_2'] == outcome) | (wills['will_3'] == outcome)]['relative_prob'].sum(), 4)

    overall_probs_df = pd.DataFrame.from_dict(
        {key: [value] for key, value in overall_probs.items()}, orient='index')
    overall_probs_df.columns = ['prob']
    overall_probs_df = overall_probs_df.sort_values('prob', ascending=False)

    return wills, overall_probs_df


def gen_plots(wills, overall_probs_df, team):
    team_name = ' '.join(word.capitalize() for word in team.split('_'))

    sns.set_style('darkgrid')
    sns.set_palette('pastel')

    plt.figure(figsize=(16, 13))
    sns.barplot(data=overall_probs_df, x='prob',
                y=overall_probs_df.index, orient='h')
    plt.yticks(rotation=22.5, fontsize='20')
    plt.xlabel('Probability of Outcome', fontsize='20')
    plt.ylabel('Outcome', fontsize='20')
    plt.title(
        f'{team_name}: Overall Likelihood of Individual Will', fontsize='25')
    plt.xticks(
        [.01, .05, .10, .15, .20, .25, .30, .35, .40, .45, .50, .60, .70, .80, .90, 1])
    plt.xlim(0, 1)
    plt.savefig(f'./plots/{team}/Wills_Individual_{team}.png', format='png')
    plt.clf()

    plt.figure(figsize=(16, 13))
    sns.barplot(data=wills, x='relative_prob',
                y=wills['will_combo'], orient='h')
    plt.yticks(rotation=22.5, fontsize='20')
    plt.xlabel('Probability of Outcome', fontsize='20')
    plt.ylabel('Outcome', fontsize='20')
    plt.title(f'{team_name}: Overall Likelihood of Will Pair', fontsize='25')
    plt.xticks(
        [.01, .05, .10, .15, .20, .25, .30, .35, .40, .45, .50, .60, .70, .80, .90, 1])
    plt.xlim(0, 1)
    plt.savefig(f'./plots/{team}/Will_Pairs_{team}.png', format='png')
    plt.clf()

    plt.figure(figsize=(16, 13))
    sns.barplot(data=wills.iloc[1:, :], x='relative_prob',
                y=wills['will_combo'][1:], orient='h')
    plt.yticks(rotation=22.5, fontsize='20')
    plt.xlabel('Probability of Outcome (Scale: 0-20%)', fontsize='20')
    plt.ylabel('Outcome', fontsize='20')
    plt.title(
        f'{team_name}: Overall Likelihood of Non-Guide Will Pairs (Zoom-in)', fontsize='25')
    plt.xticks([.01, .05, .10, .15, .20])
    plt.xlim(0, .2)
    plt.savefig(
        f'./plots/{team}/Will_Pairs_Off_Plan_{team}.png', format='png')
    plt.clf()

In [13]:
def main():

    all_draws = []

    for i, will_dict in enumerate([value for key, value in wills_values.items() if key == 'magic']):
        print(f'Team {i+1} of {len(wills_values.values())}')
        all_draws.append(free_will_sim(will_dict))
        
    for i, team in enumerate([file for file in wills_files if file == 'magic']):
        wills, overall_probs_df = get_tables(all_draws[i])

        gen_plots(wills, overall_probs_df, team)

        wills.to_csv(f'./data/{team}_wills.csv', index=None)

In [14]:
main()

Team 1 of 24
Simulation 12500...
Simulation 25000...
Simulation 37500...
Simulation 50000...


<Figure size 1152x936 with 0 Axes>

<Figure size 1152x936 with 0 Axes>

<Figure size 1152x936 with 0 Axes>